In [ ]:
# Install required libraries
!pip install numpy pandas matplotlib seaborn scikit-learn keras tensorflow opencv-python PuLP


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense


In [ ]:
# Generate synthetic retail price data
dates = pd.date_range(start="2015-01-01", periods=100, freq="M")
prices = np.random.normal(loc=500000, scale=50000, size=100)
df = pd.DataFrame({'date': dates, 'price': prices})
df.set_index('date', inplace=True)

# Normalize prices
scaler = MinMaxScaler()
df['scaled_price'] = scaler.fit_transform(df[['price']])

# Create LSTM sequences
def create_sequences(data, seq_len=12):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return np.array(X), np.array(y)

X, y = create_sequences(df['scaled_price'].values, 12)
X = X.reshape((X.shape[0], X.shape[1], 1))


In [ ]:
model = Sequential([
    LSTM(64, input_shape=(X.shape[1], X.shape[2]), return_sequences=False),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=20, batch_size=8, verbose=1)

preds = model.predict(X)
preds_rescaled = scaler.inverse_transform(preds)

plt.figure(figsize=(10,5))
plt.plot(df.index[12:], scaler.inverse_transform(y.reshape(-1, 1)), label='Actual')
plt.plot(df.index[12:], preds_rescaled, label='Predicted')
plt.legend()
plt.title("LSTM Retail Price Forecast")
plt.show()


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import zipfile
with zipfile.ZipFile("real_estate_sample.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/images")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    '/content/images',
    target_size=(64, 64),
    batch_size=16,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    '/content/images',
    target_size=(64, 64),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)

cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn.fit(train_gen, validation_data=val_gen, epochs=5)


In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

properties = ['A', 'B', 'C']
returns = [0.12, 0.18, 0.10]
costs = [100000, 150000, 120000]
budget = 250000

model = LpProblem("Portfolio Optimization", LpMaximize)
x = LpVariable.dicts("Invest", properties, cat='Binary')

model += lpSum([returns[i] * x[properties[i]] for i in range(len(properties))])
model += lpSum([costs[i] * x[properties[i]] for i in range(len(properties))]) <= budget

model.solve()

for p in properties:
    print(f"Invest in {p}: {x[p].varValue}")
